# pyFR Demo

In [14]:
import sys
from ptsa.data.readers import BaseEventReader
from ptsa.data.readers.TalReader import TalReader
from ptsa.data.readers import EEGReader

To read behavioral events we need to first localize the directory where the events are stored. If you are running the code on rhino then pyFR events are stored in /data/events/pyFR. Since I mounted rhino filesystem remotely I have to prepend /Volumes/rhino_root to this path - in my case /Volumes/rhino_root specifies the folder where I mounted rhino file system. Once you know the location you simply ocnstruct the path to the event file by using subject code (here TJ011). 

BaseEventReader takes a path to the event file and optional flag *eliminate_events_with_no_eegs* that lets you pre-filter events to include only those that have eeg data associated with them.

Finally, after you create BaseEventReader object you call its *read* function that returns a Numpy recarray with behavioral events

In [15]:
ev_path = '/Volumes/rhino_root/data/events/pyFR/TJ011_events.mat'
base_e_reader = BaseEventReader(filename=ev_path, eliminate_events_with_no_eeg=True)
base_events = base_e_reader.read()

In [16]:
base_events = base_events[base_events.type == 'WORD']

To examine the conten of the event recarray we can print its columns one by one and do some filtering that matches our criteria. For example let's print all the columns of the recarray

In [17]:
base_events.dtype

dtype((numpy.record, [('subject', 'S256'), ('session', '<i8'), ('list', '<i8'), ('serialpos', '<i8'), ('type', 'S256'), ('item', 'S256'), ('itemno', '<i8'), ('recalled', '<i8'), ('mstime', '<f8'), ('msoffset', '<i8'), ('rectime', '<i8'), ('intrusion', '<i8'), ('eegfile', 'S256'), ('eegoffset', '<i8')]))

Above you can see a summary of the fields that are available in the record array. Let's print first 2 events to see actual content of the record array:

In [18]:
base_events[:2]

rec.array([ ('TJ011', 0, 1, 1, 'WORD', 'CUBE', 72, 0, 1270575931057.0, 1, -999, -999, '/Volumes/rhino_root/data/eeg/TJ011/eeg.noreref/TJ011_06Apr10_1319', 235074),
 ('TJ011', 0, 1, 2, 'WORD', 'JUDGE', 142, 0, 1270575933544.0, 1, -999, -999, '/Volumes/rhino_root/data/eeg/TJ011/eeg.noreref/TJ011_06Apr10_1319', 235572)], 
          dtype=[('subject', 'S256'), ('session', '<i8'), ('list', '<i8'), ('serialpos', '<i8'), ('type', 'S256'), ('item', 'S256'), ('itemno', '<i8'), ('recalled', '<i8'), ('mstime', '<f8'), ('msoffset', '<i8'), ('rectime', '<i8'), ('intrusion', '<i8'), ('eegfile', 'S256'), ('eegoffset', '<i8')])

As you can tell from the output above each event contains subject, session number, list number, serial position number, type of the event (here all events are WORD events because we have performed filtering base_events = base_events[base_events.type == 'WORD']) etc.... Let's see what words were displayed on the screen:

In [19]:
base_events["item"][:20]

array(['CUBE', 'JUDGE', 'BRUSH', 'HILL', 'STEM', 'HOUSE', 'HORSE', 'ROCK',
       'HOOF', 'GEESE', 'APE', 'YARD', 'TANK', 'HOOK', 'CROW', 'PIT',
       'PLATE', 'GIRL', 'DUCK', 'BOMB'], 
      dtype='|S256')

base_events["item"] is an array of all the words displayed during entire experiment. Since I wanted to display only first 20 words I used slicing operation base_events["item"][:20]

Notice that if we want to display e.g. serialpos column we can use the following syntax:

In [20]:
base_events.serialpos[:20]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,  1,  2,
        3,  4,  5])

or we could use another equivalent syntax:

In [21]:
base_events["serialpos"][:20]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,  1,  2,
        3,  4,  5])

However when we try to use the first syntax for *item* we will get into trouble:  

In [22]:
base_events.item[:20]

TypeError: 'builtin_function_or_method' object has no attribute '__getitem__'

This happens because *item* is built-in name of of the funuction belonging to recarray and by typing  *base_events.item* you are referring to the function not the column of the array. Thus to access words on the screen via *item*  you have to use base_events["item"][:20]